<img width="8%" alt="Naas.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Naas.png" style="border-radius: 15%">

# LinkedIn - Get post interactions

**Tags:** #linkedin #post #interactions #api #python #data

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-12-12 (Created: 2023-12-12)

**Description:** This notebook will show how to get post interactions from LinkedIn using the API. It will cover the setup of the API, the model to get the post interactions and the display of the result.

**References:**
- [LinkedIn API Documentation](https://docs.microsoft.com/en-us/linkedin/shared/references/v2/profile/shares?context=linkedin/consumer/context)
- [LinkedIn API Explorer](https://developer.linkedin.com/docs/rest-api/api-explorer)

## Input

### Import libraries

In [ ]:
import requests
import json

### Setup variables
- **access_token**: Access token to use the LinkedIn API. [Link to procedure](https://docs.microsoft.com/en-us/linkedin/shared/authentication/authorization-code-flow?context=linkedin/consumer/context)
- **post_id**: ID of the post to get interactions

In [ ]:
access_token = "AQXd8_4BAAAAU7mGj5yvqVESdE9jT2djQX_VuBdGf7KW0HxI"
post_id = "123456789"

## Model

### Get post interactions

This function will get the interactions of a post from LinkedIn using the API.

In [ ]:
def get_post_interactions(access_token, post_id):
    # Setup the request
    url = f"https://api.linkedin.com/v2/socialActions/{post_id}/interactions"
    headers = {"Authorization": f"Bearer {access_token}"}
    # Make the request
    response = requests.get(url, headers=headers)
    # Check the response
    if response.status_code == 200:
        # Return the response
        return response.json()
    else:
        # Return an error
        return {"error": response.status_code}

## Output

### Display result

In [ ]:
# Get post interactions
post_interactions = get_post_interactions(access_token, post_id)
# Display the result
print(json.dumps(post_interactions, indent=4))